# Basic Usage Example

This notebook demonstrates the basic usage of `claude-message-visualizer`.

## Setup

**Recommended:** Create a `.env` file in your project root:
```
ANTHROPIC_API_KEY=your-api-key-here
```

This file is already in `.gitignore` so your key won't be committed to git.

The code below uses `python-dotenv` to load the key securely from your `.env` file.

In [6]:
# Install if needed
# !pip install claude-message-visualizer anthropic python-dotenv

In [7]:
from dotenv import load_dotenv
from anthropic import Anthropic
from claude_message_visualizer import visualize_message

load_dotenv()
client = Anthropic()

## Example 1: Simple Text Response

In [8]:
response = client.messages.create(
    model="claude-sonnet-4-5",
    max_tokens=100,
    messages=[{
        "role": "user",
        "content": "Hello, Claude!"
    }]
)

visualize_message(response)

## Example 2: Longer Response with Auto-collapse

In [9]:
response = client.messages.create(
    model="claude-sonnet-4-5",
    max_tokens=2000,
    messages=[{
        "role": "user",
        "content": "Write a detailed explanation of how neural networks work, including backpropagation."
    }]
)

# Long text blocks (>1000 chars) are automatically collapsible
visualize_message(response)

## Example 3: Extended Thinking

This example shows how the visualizer displays Claude's internal reasoning process. The thinking block is collapsible and shows the character count.

In [10]:
response = client.messages.create(
    model="claude-sonnet-4-5",
    max_tokens=8000,
    thinking={
        "type": "enabled",
        "budget_tokens": 5000
    },
    messages=[{
        "role": "user",
        "content": "Design a spam detection system using machine learning. What are the key challenges?"
    }]
)

visualize_message(response)

## Example 4: Tool Use

This example shows how the visualizer displays tool use blocks. When Claude wants to use a tool, it creates a `tool_use` block with the tool name and formatted JSON input parameters.

In [ ]:
from anthropic.types import ToolParam

tools: list[ToolParam] = [{
    "name": "get_weather",
    "description": "Get current weather for a location",
    "input_schema": {
        "type": "object",
        "properties": {
            "location": {"type": "string", "description": "City name"}
        },
        "required": ["location"]
    }
}]

response = client.messages.create(
    model="claude-sonnet-4-5",
    max_tokens=1000,
    tools=tools,
    messages=[{"role": "user", "content": "What's the weather in San Francisco?"}]
)

visualize_message(response)

## Example 5: Web Search with Citations

This example shows the visualizer's citation feature. When Claude uses web search and cites sources, the visualizer displays:
- Web search results as collapsible cards
- Citations with red-highlighted quoted text
- Automatic citation numbering linked to search results

In [13]:
response = client.messages.create(
    model="claude-sonnet-4-5",
    max_tokens=2000,
    tools=[{"type": "web_search_20250305", "name": "web_search", "max_uses": 5}],
    messages=[{
        "role": "user",
        "content": "What are the latest recommendations for vitamin D intake? Please cite your sources."
    }]
)

visualize_message(response)

## Next Steps

Now that you've seen the visualizer's features, try:
- Modify the examples above with your own prompts
- Experiment with different tool definitions
- Combine features (e.g., extended thinking + web search)

For more information:
- [Full Documentation](../README.md)
- [Report Issues](https://github.com/frankiewarren/claude-message-visualizer/issues)
- [Claude API Docs](https://docs.anthropic.com/)